# A standard up-down method  for measuring mechanical nociception in rodents using von Frey filaments

## Absstract
An exaggerated nocifensive response to mechanical stimuli is considered a key indicator of abnormal sensory processing in most rodent models of pathological pain. Mechanosensitivity can be determined as the threshold amount of force required to elicit a behavioral response, such as the withdrawal of a paw from the applied stimulus. There are different methods available to estimate paw withdrawal threshold (PWT) with the use of von Frey filaments, but the up-down method of Dixon as applied to rodents by Chaplan et al. remains one of the most commonly used. 

## Introduction
In the method of ***Chaplan*** et al., the threshold is considered to have been reached the first time that the response of the animal to a von Frey filament is different from the previous trial, after which four additional von Frey presentations are conducted in an up-down manner. At the end of the trial the paw withdrawal estimate (PWT) is calculated by taking the value of the final filament used and adding an adjustment factor derived from the response pattern using the calculations of Dixon. 

![1744-8069-10-26-6-2.jpg](attachment:1744-8069-10-26-6-2.jpg)

For this tutorial, we developed a simple method for estimating paw withdrawal threshold (PWT) with von Frey filaments that uses Chaplan method. We refined the PWT calculation to allow the estimation of PWT with adjustment factor from a look-up tables of Dixon.

PWT was converted from filament number to force, and expressed in grams using the equation:
$$
PWT_f = 10^{(DF+B)}
$$
where F is the djustment factor from a look-up tables using the calculations of Dixon; D is the differnce estimates measured filament bending force plotted against the filament number using the; B is the force of the last von Frey tested.

## Set-up 

In [ ]:
import numpy as np
import pandas as pd

## Get last von Frey tested

In [ ]:
# This is the von Frey used
vf2 = [0.38, 0.57, 1.23, 1.83, 3.66, 5.93, 9.13, 13.1]
vf = np.array(vf2)
lvf2 = np.log10(vf2)
# determine difference
kvf2 = lvf2[1:len(lvf2)] - lvf2[0:(len(lvf2) - 1)]
mk2 = np.mean(kvf2)
d = mk2

In [ ]:
# get last von Frey tested
# assumes that the starting point for the testing
# is the fourth member of the von Frey group
def get_last(x):
    # x = x.str.lower()
    plc = 3
    for i in range(0, len(x) - 1):
        if x[i] == "o":
            plc += 1
        if x[i] == "x":
            plc -= 1
    return vf2[plc]

## Estimating paw withdrawal threshold

In [ ]:
# read the test result
df = pd.read_csv('PWT_test_result.csv', index_col=None)
df.head()

,No.,Response
0,1.0,ooooo
1,2.0,oooxxxox
2,3.0,ooooo
3,4.0,ooooo
4,5.0,Could not match sequence


In [ ]:
# get the last force in the each test result
df['Last'] = df['Response'].apply(get_last)
df.head()

,No.,Response,Last
0,1.0,ooooo,13.10
1,2.0,oooxxxox,3.66
2,3.0,ooooo,13.10
3,4.0,ooooo,13.10
4,5.0,Could not match sequence,5.93


In [ ]:
table_df = pd.read_csv('factor_looup_table.csv', index_col=None)
table_df.head()

,Response,Value
0,ox,-0.500
1,oox,-0.388
2,ooox,-0.378
3,oooox,-0.377
4,oxo,0.842


In [ ]:
df_merged = pd.merge(df, table_df, how='left')
df_merged.head()

,No.,Response,Last,Value
0,1.0,ooooo,13.10,1.000
1,2.0,oooxxxox,3.66,0.244
2,3.0,ooooo,13.10,1.000
3,4.0,ooooo,13.10,1.000
4,5.0,Could not match sequence,5.93,NaN


In [ ]:
# calculates the PWT by taking the log base 10 of the last von Frey used
# and multiplying it but the difference and k
pwtl = np.log10(df_merged['Last']) + d * df_merged['Value']
df_merged['Force'] = 10**pwtl
df_merged.head()

,No.,Response,Last,Value,Force
0,1.0,ooooo,13.10,1.000,21.722630
1,2.0,oooxxxox,3.66,0.244,4.140698
2,3.0,ooooo,13.10,1.000,21.722630
3,4.0,ooooo,13.10,1.000,21.722630
4,5.0,Could not match sequence,5.93,NaN,NaN


## Save the result

In [ ]:
df_merged.to_csv('Result.csv')